In [1]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial



code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel, PerturbedBPRMLPModel
from make_datasets import make_data


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from bpr import bpr_variable_k_no_ties

2023-06-27 04:34:07.084783: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 04:34:07.087120: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 04:34:07.131965: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 04:34:07.133026: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 04:34:08.846231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [43]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_annual_tract/'
seed = 360
lookback_years=5
learning_rate=0.01
add_spacetime=True
add_svi=True

time_window = lookback_years
timesteps_per_year = 1
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020


tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['deaths']

if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


#data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=timesteps_per_year)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=timesteps_per_year)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=timesteps_per_year)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)
from zinf_gp.metrics import normcdf, fixed_top_X

In [44]:
linear_poisson = sklearn.linear_model.PoissonRegressor()

In [45]:
features_only

['deaths',
 'lat',
 'lon',
 'timestep',
 'theme_1_pc',
 'theme_2_pc',
 'theme_3_pc',
 'theme_4_pc',
 'svi_pctile']

In [46]:
train_x_long = tf.reshape(train_x_BSF_flat,((lookback_years*1620),lookback_years*len(features_only)))
train_y_long = tf.reshape(train_y_BS,(lookback_years*1620,1))

In [47]:
reg = linear_poisson.fit(train_x_long, tf.squeeze(train_y_long))

In [48]:
pred_linear_2019 = reg.predict(test_x_BSF_flat[0])
bpr_variable_k_no_ties(test_y_BS[0], pred_linear_2019,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.58760685>

In [49]:
pred_linear_2020 = reg.predict(test_x_BSF_flat[1])
bpr_variable_k_no_ties(test_y_BS[1], pred_linear_2020,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.61723447>

In [50]:
(0.58760685+0.61723447)/2

0.6024206599999999

In [51]:
(0.5982906+0.6032064)/2

0.6007485

In [52]:
np.mean([mean_absolute_error(test_y_BS[0], pred_linear_2019),
mean_absolute_error(test_y_BS[1], pred_linear_2020)])

1.052748407663083

In [53]:
np.mean([sqrt(mean_squared_error(test_y_BS[0], pred_linear_2019)),
sqrt(mean_squared_error(test_y_BS[1], pred_linear_2020))])

1.3809252248671406

In [56]:
from sklearn.ensemble import HistGradientBoostingRegressor
max_valid=0
for depth in [3,6,9]:
    for leaf in [2,5,10]:
        for sampl in [50,100,400]:
            hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=depth, max_leaf_nodes=leaf,
                                                           l2_regularization=0.1, min_samples_leaf=sampl )
            hist_reg = hist_poisson.fit(train_x_long, tf.squeeze(train_y_long))
            
            valid_fit = hist_reg.predict(valid_x_BSF_flat[0])
            new_valid = bpr_variable_k_no_ties(valid_y_BS[0], valid_fit,k=100)
            
            if new_valid>max_valid:
                print(f'{depth}, {leaf}, {sampl}')
                max_valid=new_valid
                print(max_valid)

3, 2, 50
tf.Tensor(0.5782881, shape=(), dtype=float32)


KeyboardInterrupt: 

In [57]:
hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=3, max_leaf_nodes=2,
                                                           l2_regularization=1, min_samples_leaf=100 )
hist_reg = hist_poisson.fit(train_x_long, tf.squeeze(train_y_long))

In [58]:
pred_hist_2019 = hist_reg.predict(test_x_BSF_flat[0])
bpr_variable_k_no_ties(test_y_BS[0], pred_hist_2019,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5534188>

In [59]:
pred_hist_2020 = hist_reg.predict(test_x_BSF_flat[1])
bpr_variable_k_no_ties(test_y_BS[1], pred_hist_2020,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5751503>

In [60]:
train_fit = hist_reg.predict(train_x_BSF_flat[0])
bpr_variable_k_no_ties(train_y_BS[0], train_fit,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.48398575>

In [61]:
valid_fit = hist_reg.predict(valid_x_BSF_flat[0])
bpr_variable_k_no_ties(valid_y_BS[0], valid_fit,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5803758>

In [62]:
(0.5751503+0.5534188)/2

0.56428455

In [68]:
np.mean([sqrt(mean_squared_error(test_y_BS[0], pred_hist_2019)),
sqrt(mean_squared_error(test_y_BS[1], pred_hist_2020))])

1.20849009336153

In [70]:
np.mean([mean_absolute_error(test_y_BS[0], pred_hist_2019),
mean_absolute_error(test_y_BS[1], pred_hist_2020)])

0.9019008909597088

In [217]:
bpr_variable_k_no_ties(valid_y_BS[0], valid_fit*0+1,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.217119>

In [220]:
perturbed_top_100 = perturbed(top_100_idx_func,
                              num_samples=5000,
                              sigma=0.05,
                              noise='normal',
                              batched=True)


In [225]:
fixed_top_X( multiindexed_gdf[(multiindexed_gdf['year']==2019)]['deaths'], 
            multiindexed_gdf[(multiindexed_gdf['year']==2019)]['deaths']*0,X=100)

(468.0, 0.0, 352.975, 0.24577991452991454)

In [226]:
fixed_top_X( multiindexed_gdf[(multiindexed_gdf['year']==2020)]['deaths'], 
            multiindexed_gdf[(multiindexed_gdf['year']==2020)]['deaths']*0,X=100)

(499.0, 0.0, 378.734, 0.24101402805611225)

In [227]:
(0.24577991452991454+0.24101402805611225)/2

0.2433969712930134

In [12]:
tf.reduce_sum(tf.sort(test_y_BS[0])[-100:])

<tf.Tensor: shape=(), dtype=float32, numpy=468.0>

In [13]:
(468+499)/2

483.5

In [15]:
483.5*0.004

1.934

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
